In [1]:
import gzip
import numpy as np
import os
import mnist_tools
import tensorflow as tf
tf.__version__

'1.6.0-rc1'

In [2]:
DATA_DIR = "/var/ellie/data/mnist"
img_train = os.path.join(DATA_DIR, "train-images-idx3-ubyte.gz")
lbl_train = os.path.join(DATA_DIR, "train-labels-idx1-ubyte.gz")
img_test = os.path.join(DATA_DIR, "t10k-images-idx3-ubyte.gz")
lbl_test = os.path.join(DATA_DIR, "t10k-labels-idx1-ubyte.gz")
os.listdir(DATA_DIR)

['t10k-images-idx3-ubyte.gz',
 't10k-labels-idx1-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 'train-images-idx3-ubyte.gz']

In [23]:
def _scale(imgs, lbls):
    return imgs / 256, lbls

BATCH_SIZE=1000
tr, te = mnist_tools.datasets(DATA_DIR)

In [24]:
train_iter = tr.repeat().batch(BATCH_SIZE).make_one_shot_iterator().get_next()
tr_images, tr_labels = train_iter
tr_images

<tf.Tensor 'IteratorGetNext_1:0' shape=(?, 784) dtype=uint8>

In [5]:
with tf.Session() as sess:
    i,l = sess.run([tr_images, tr_labels])
    print(i[0, 290:300])
i.shape, l.shape

[  1 154 253  90   0   0   0   0   0   0]


((10000, 784), (10000, 10))

In [32]:
def Dense( hidden ):
    def _dense ( X, Y_ ):
        #X=tf.placeholder(shape=[None, 784], dtype=tf.float32)
        #Y_=tf.placeholder(shape=[None, 10], dtype=tf.float32)
        X = tf.cast(X, tf.float32)
        X = tf.subtract(tf.divide(X, tf.constant(256.0)), tf.constant(0.5))
        W1 = tf.Variable(tf.random_normal(shape=[784, hidden]), dtype=tf.float32)
        b1 = tf.Variable(tf.random_normal(shape=[hidden]), dtype=tf.float32)
        a1 = tf.nn.relu(tf.matmul(X, W1) + b1)
        W2 = tf.Variable(tf.random_normal(shape=[hidden, 10]), dtype=tf.float32)
        b2 = tf.Variable(tf.random_normal(shape=[10]), dtype=tf.float32)
        logits = tf.matmul(a1, W2) + b2
        prob = tf.nn.softmax(logits)
        pred = tf.argmax(prob, axis=1)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=logits))
        optimizer = tf.train.AdamOptimizer()
        train_op = optimizer.minimize(loss)
        accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(Y_, axis=1), pred), dtype=tf.float32))
        return (X, Y_, train_op, loss, accuracy)
    return _dense


In [33]:
X, Y_, train_op, loss, accuracy = Dense(128)(tr_images, tr_labels)
init = tf.global_variables_initializer()

In [35]:
number_of_epochs = 100
number_of_steps = number_of_epochs * 60000 // BATCH_SIZE
report_every_after = 100

print("number of epochs  : %s " % number_of_epochs)
print("batch size        : %s " % BATCH_SIZE)
print("report every after: %s steps" % report_every_after)

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(number_of_steps):
        try:
            _, _loss, _accuracy = sess.run([train_op, loss, accuracy])
            if i % report_every_after == 0:
                print('Batch No %s: accuracy: %s, loss: %s' % (i, _accuracy, _loss))
        except tf.errors.OutOfRangeError as e:
            print("Done.")
            

number of epochs  : 100 
batch size        : 1000 
report every after: 100 steps
Batch No 0: accuracy: 0.13, loss: 141.9151
Batch No 100: accuracy: 0.532, loss: 16.102262
Batch No 200: accuracy: 0.679, loss: 9.837561
Batch No 300: accuracy: 0.769, loss: 5.5614495
Batch No 400: accuracy: 0.791, loss: 4.070766
Batch No 500: accuracy: 0.799, loss: 4.790814
Batch No 600: accuracy: 0.853, loss: 3.1983514
Batch No 700: accuracy: 0.863, loss: 2.29559
Batch No 800: accuracy: 0.844, loss: 3.3503318
Batch No 900: accuracy: 0.884, loss: 2.3239386
Batch No 1000: accuracy: 0.886, loss: 1.5484669
Batch No 1100: accuracy: 0.869, loss: 2.5623999
Batch No 1200: accuracy: 0.902, loss: 1.8710258
Batch No 1300: accuracy: 0.903, loss: 1.1629597
Batch No 1400: accuracy: 0.887, loss: 2.0413601
Batch No 1500: accuracy: 0.911, loss: 1.5472491
Batch No 1600: accuracy: 0.916, loss: 0.9090353
Batch No 1700: accuracy: 0.913, loss: 1.6893337
Batch No 1800: accuracy: 0.922, loss: 1.2785798
Batch No 1900: accuracy: 0